<a href="https://colab.research.google.com/github/RafsanJany-44/ARC/blob/master/Fitbit_Main_codebase_Dailyactivity_Pot_Initial_02_07_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from tqdm.notebook import tqdm
from glob import glob
import os
import pandas as pd
import ipywidgets as widgets



def get_content(patient_folder_adrs):

  for i in glob(patient_folder_adrs+"/*"):
    content = i.split("_")
    if "heartrate" in content and "seconds" in content:
      post_sec = i
    if "minuteMETsNarrow" in content:
      post_met = i

  return post_sec,post_met



def items_check_box(items):
  new_keys=[]
  for i in items:
    i=widgets.Checkbox(
      value=False,
      description=str(i),
      disabled=False,
      indent=False
      )
    display(i)
    new_keys.append(i)
  return new_keys
result={}


def week_count(df,time_col):
    df[time_col] = pd.to_datetime(df[time_col])
    df['Date'] = df[time_col].dt.date
    df[time_col] = df[time_col].dt.strftime('%m/%d/%Y %I:%M:%S %p')
    df[time_col] = df[time_col].astype(str)
    min_date = df['Date'].min()
    df['Week_Number'] = (df['Date'] - min_date).dt.days // 7 + 1
    return df






In [3]:
path = "/content/drive/MyDrive/Iqram Sir/Fitbit_temp/data"



#path = "/home/iqh4001/Iqram_WCM/MADE Lab/Wearable Project/Data/Fitbit Database/ Dr. Kaur's Lab/Aim 3 Fitbit Data"

In [4]:
sub_list=[]

for i in sorted(glob(path+"/*")):
  sub_list.append(os.path.basename(i))

new_keys=items_check_box(sub_list)

Checkbox(value=False, description='Sub_1', indent=False)

Checkbox(value=False, description='Sub_2', indent=False)

Checkbox(value=False, description='Sub_3', indent=False)

In [5]:
subject_list = []
for i in range(len(new_keys)):
  if new_keys[i].value ==True:
    subject_list.append(sub_list[i])

subject_list

['Sub_1']

In [6]:





for i in subject_list:
    datapath= path+'/'+i
    Post_sec,Post_met = get_content(datapath)
    df_post_sec_ini = week_count(pd.read_csv(Post_sec),'Time')
    df_post_met_ini = week_count(pd.read_csv(Post_met),'ActivityMinute')




In [13]:
df_post_sec_ini

,Time,Value,Date,Week_Number
0,01/13/2023 11:14:17 AM,97,2023-01-13,1
1,01/13/2023 11:14:39 AM,104,2023-01-13,1
2,01/13/2023 11:14:44 AM,106,2023-01-13,1
3,01/13/2023 11:14:49 AM,103,2023-01-13,1
4,01/13/2023 11:14:54 AM,94,2023-01-13,1
...,...,...,...,...
978423,07/14/2023 09:29:54 AM,102,2023-07-14,27
978424,07/14/2023 09:29:59 AM,101,2023-07-14,27
978425,07/14/2023 09:30:09 AM,99,2023-07-14,27
978426,07/14/2023 09:30:14 AM,97,2023-07-14,27


In [12]:
df_post_sec_ini['Time']

0         01/13/2023 11:14:17 AM
1         01/13/2023 11:14:39 AM
2         01/13/2023 11:14:44 AM
3         01/13/2023 11:14:49 AM
4         01/13/2023 11:14:54 AM
                   ...          
978423    07/14/2023 09:29:54 AM
978424    07/14/2023 09:29:59 AM
978425    07/14/2023 09:30:09 AM
978426    07/14/2023 09:30:14 AM
978427    07/14/2023 09:30:24 AM
Name: Time, Length: 978428, dtype: object

In [ ]:
# Convert the pickup time to datetime and create a new column truncated to day.

df['pickup'] = pd.to_datetime(df.pickup)
df['pickup_date'] = pd.to_datetime(df.pickup.dt.date)
df = df.loc[df['pickup_date'] >= '2019-03-01'][:5000]

# We also create a grouped version, with calculated mean and standard deviation.
df_grouped = (
    df[['pickup_date', 'fare']].groupby(['pickup_date'])
    .agg(['mean', 'std', 'count'])
)
df_grouped = df_grouped.droplevel(axis=1, level=0).reset_index()
# Calculate a confidence interval as well.
df_grouped['ci'] = 1.96 * df_grouped['std'] / np.sqrt(df_grouped['count'])
df_grouped['ci_lower'] = df_grouped['mean'] - df_grouped['ci']
df_grouped['ci_upper'] = df_grouped['mean'] + df_grouped['ci']
df_grouped.head()